In [1]:
from __future__ import print_function
import time
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from matplotlib.colors import LogNorm
import umap.umap_ as uma
import math

In [47]:
from Execute116 import *
from plotting import *
import math
# from Convert import *

In [3]:
sys.path.insert(0, r'C:\Users\viska\Documents\AceCan')
os.chdir(r"C:\Users\viska\Documents\AceCan")
Output_path = r"C:\Users\viska\Documents\AceCan\Prepros"
# os.mkdir(Output_path)
load_data = 1
num = 40
base = 8
topk = 2000
col_range =  [8000]
row_range = [5]
display = 20
pca_comp = 8
d = base ** pca_comp  
data_dir = r'.\bki'    
test =1
name = os.path.join(Output_path, f'b{base}topk{topk}N{num}test{test}')
idx_plot = 3
# os.mkdir(name)
if test:
    pixel = 400*300
else:
    pixel = 1004 * 1344
# os.mkdir(name)

In [61]:
exact_pdh = pd.read_csv(f'{name}\\exact_pdh.cvs', names = list(range(8))+ ['freq','val','u1','u2','t1','t2','idx'], sep = ' ')
freqs = exact_pdh['freq']
vals = exact_pdh['val']


In [62]:
exact_pdh

,0,1,2,3,4,5,6,7,freq,val,u1,u2,t1,t2,idx
0,0.0,5.0,4.0,4.0,4.0,3.0,3.0,4.0,49875.0,9292072.0,-1.898836,7.282517,-3.417665,2.826575,0.0
1,0.0,5.0,3.0,3.0,4.0,4.0,3.0,4.0,39918.0,9324264.0,-3.093164,6.554080,2.497411,4.317867,1.0
2,0.0,5.0,4.0,3.0,4.0,4.0,3.0,4.0,32220.0,9324328.0,-2.062715,7.365609,-3.223826,1.445663,2.0
3,0.0,4.0,3.0,3.0,4.0,4.0,3.0,4.0,28804.0,9324256.0,-2.021802,4.798568,2.855695,2.550706,3.0
4,0.0,4.0,3.0,4.0,3.0,3.0,3.0,4.0,28127.0,9287904.0,-1.692642,5.286952,2.383340,0.990132,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,0.0,5.0,2.0,2.0,3.0,3.0,3.0,4.0,519.0,9286824.0,-4.416726,5.469489,6.031096,5.045776,599.0
600,0.0,2.0,2.0,4.0,3.0,4.0,3.0,4.0,513.0,9320592.0,-1.052679,2.693677,-4.351082,-7.836273,600.0
601,0.0,4.0,5.0,3.0,4.0,4.0,3.0,3.0,508.0,7227232.0,-0.780493,7.013070,0.987039,-1.959625,601.0
602,0.0,3.0,3.0,4.0,2.0,3.0,3.0,4.0,505.0,9283800.0,-0.786497,4.835190,-7.313774,-3.955187,602.0


In [48]:
def check_cluster(diff, base):
    #TODO: check if we need to limit decode
    if diff == 0:
        return True
    else:
        decode = math.log(abs(diff), base)
        same_cluster_query = np.floor(decode) == decode           
        return same_cluster_query

In [79]:
def check_cluster(diff, base):
    #TODO: check if we need to limit decode
    if diff == 0:
        return True
    else:
        decode = math.log(abs(diff), base)
        same_cluster_query = np.floor(decode) == decode   
        return same_cluster_query
        
def get_merged_HHs(vals, limit = 10):
    val_l = len(vals)
    clusters = np.zeros(val_l)
    processed = [False] * val_l
    cluster_dict= {}
    color_idx = 1 
    for idx_1 in range(val_l):
        if not processed[idx_1]:
            val = vals[idx_1]
            cluster_dict[f'color_{color_idx}'] = [val]
            clusters[idx_1] = color_idx
            for idx_2 in range(idx_1+1, val_l): 
                val_2 = vals[idx_2]
                diff = abs(val - val_2)
                if check_cluster(diff,8):
                    clusters[idx_2] = color_idx
                    cluster_dict[f'color_{color_idx}'] += [val_2]
                    processed[idx_2] = True
            color_idx += 1  
            processed[idx_1]  = True
        if color_idx > limit:
            print(f'Top {limit} color clusters after merger found, to continue increase limit' )
            break
    return clusters, cluster_dict

In [82]:
clusters, cluster_dict = get_merged_HHs(vals, limit = 10)
clusters

Top 10 color clusters after merger found, to continue increase limit


array([ 1.,  2.,  2.,  2.,  3.,  4.,  5.,  3.,  1.,  6.,  1.,  2.,  3.,
        7.,  8.,  3.,  9.,  6.,  1.,  1., 10.,  0.,  4.,  6.,  0.,  1.,
        7.,  0., 10.,  3.,  5.,  7.,  0.,  0., 10.,  0.,  5.,  0.,  0.,
        4.,  6.,  6.,  4.,  0.,  0.,  0.,  5.,  0.,  0.,  0.,  0.,  7.,
        0.,  0.,  0.,  2.,  0.,  0.,  4.,  0.,  8.,  0.,  1.,  0.,  8.,
        0.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0., 10.,  0.,  0.,  0.,  0.,  0.,  7.,  0.,  0.,  0.,
        0.,  4.,  0.,  8.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0., 10.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.,  9.,  0.,  0.,  0.,
        0.,  0.,  0.,  0., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
       10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  7.,
        0.,  0.,  6.,  0.,  0.,  0.,  0., 10.,  0.,  9.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [83]:
exact_pdh['cluster'] = clusters

In [84]:
exact_pdh


,0,1,2,3,4,5,6,7,freq,val,u1,u2,t1,t2,idx,cluster
0,0.0,5.0,4.0,4.0,4.0,3.0,3.0,4.0,49875.0,9292072.0,-1.898836,7.282517,-3.417665,2.826575,0.0,1.0
1,0.0,5.0,3.0,3.0,4.0,4.0,3.0,4.0,39918.0,9324264.0,-3.093164,6.554080,2.497411,4.317867,1.0,2.0
2,0.0,5.0,4.0,3.0,4.0,4.0,3.0,4.0,32220.0,9324328.0,-2.062715,7.365609,-3.223826,1.445663,2.0,2.0
3,0.0,4.0,3.0,3.0,4.0,4.0,3.0,4.0,28804.0,9324256.0,-2.021802,4.798568,2.855695,2.550706,3.0,2.0
4,0.0,4.0,3.0,4.0,3.0,3.0,3.0,4.0,28127.0,9287904.0,-1.692642,5.286952,2.383340,0.990132,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,0.0,5.0,2.0,2.0,3.0,3.0,3.0,4.0,519.0,9286824.0,-4.416726,5.469489,6.031096,5.045776,599.0,0.0
600,0.0,2.0,2.0,4.0,3.0,4.0,3.0,4.0,513.0,9320592.0,-1.052679,2.693677,-4.351082,-7.836273,600.0,0.0
601,0.0,4.0,5.0,3.0,4.0,4.0,3.0,3.0,508.0,7227232.0,-0.780493,7.013070,0.987039,-1.959625,601.0,0.0
602,0.0,3.0,3.0,4.0,2.0,3.0,3.0,4.0,505.0,9283800.0,-0.786497,4.835190,-7.313774,-3.955187,602.0,0.0


In [49]:
def run_step_merge(top_color, freq, base, limit):
    k = len(top_color)
    assert (k == len(freq))  
    processed = [False] * k
    color_dict_sketch = {}
    color_idx = 1 
    for i in range(k-1):
        if not processed[i]:
            color_i = top_color[i]
            color_dict_sketch[f'color_{color_idx}'] = [color_i]
            for j in range(i+1,k):
                color_j = top_color[j]
                diff  = color_i - color_j
                if check_cluster(diff, base):
                    processed[j] = True
                    top_color[j] = color_i
                    color_dict_sketch[f'color_{color_idx}'] += [color_j]
            #plt.matshow(np.reshape(color_map, [400,300]))
            color_idx+=1
            if color_idx > limit:
                print(f'Top {limit} color clustering after merger found, to continue increase merge_topk value' )
                break
    print(color_dict_sketch['color_1'])
    return color_dict_sketch, top_color

In [109]:
color_dict_sketch, top_color = run_step_merge(exact_val[:1000], exact_freq[:1000], base, limit = 1000)

[112344.0, 112344.0, 112344.0, 112344.0, 112344.0, 112344.0, 112344.0, 112344.0, 112344.0, 112344.0, 112344.0]


In [111]:
exact_hhpd = pd.DataFrame(exact_hhs[:1000], columns = ['val', 'freq'])
exact_hhpd['cluster'] = 0
exact_hhpd['top_color'] = top_color

In [65]:
exact_hhpd.loc[exact_hhpd['val'] == 112344, ['cluster'] ]= 11

In [107]:
exact_hhpd1 = exact_hhpd.loc[exact_hhpd['freq'] > 500]

In [121]:
exact_hhpd

,val,freq,cluster,top_color
0,112344.0,547716,1,112344.0
1,112336.0,212410,0,112344.0
2,112334.0,29164,2,112334.0
3,112280.0,17425,0,112344.0
4,112342.0,16732,0,112334.0
...,...,...,...,...
995,144628.0,68,0,144628.0
996,112770.0,68,0,112258.0
997,117041.0,68,0,117033.0
998,107750.0,68,0,111846.0


In [124]:
for i in range(1, 394):
    for val in color_dict_sketch[f'color_{i}']:
        exact_hhpd.loc[exact_hhpd['top_color'] == val, ['cluster'] ]= i
        

In [141]:
import os
os.getcwd()

'C:\\Users\\viska\\Documents\\AceCan'

In [142]:
stream = np.loadtxt(r'.\\results/b8topk1000N1exact_test/stream_concat.csv')

In [145]:
stream_pca = inverse_mapcode( stream, base, n_comp)

In [147]:
stream_pd = pd.DataFrame(stream_pca.T, columns = range(6))

In [148]:
stream_pd['org'] = stream

In [150]:
np.savetxt('streampd.csv', stream_pd)

In [117]:
exact1000 = inverse_mapcode(exact_val[:1000], base, n_comp)

In [119]:
np.shape(exact1000)

(6, 1000)